# 🔒 Security 模块测试

本 notebook 用于测试项目中 `app/core/security.py` 模块的功能。

## 目录

1. [环境配置](#环境配置)
2. [密码哈希测试](#密码哈希测试)
3. [密码验证测试](#密码验证测试)
4. [JWT 令牌生成测试](#jwt-令牌生成测试)
5. [完整认证流程演示](#完整认证流程演示)

## 🛠️ 环境配置

In [4]:
import os
import sys

# 设置项目根路径，使其能够导入项目模块
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print(f"📁 项目根路径: {PROJECT_ROOT}")

📁 项目根路径: /home/tomy/blog_fr/backend


In [5]:
# 导入项目的 security 模块
from app.core.security import (
    create_access_token,
    verify_password,
    get_password_hash,
    ALGORITHM
)
from app.core.config import settings
from datetime import timedelta
import jwt

print("✅ 成功导入 security 模块")
print(f"🔑 使用的算法: {ALGORITHM}")
print(f"⏰ 默认 Token 过期时间: {settings.ACCESS_TOKEN_EXPIRE_MINUTES} 分钟")

✅ 成功导入 security 模块
🔑 使用的算法: HS256
⏰ 默认 Token 过期时间: 10080 分钟


## 🔐 密码哈希测试

测试 `get_password_hash()` 函数，该函数使用 bcrypt 算法对密码进行哈希。

In [3]:
# 测试密码哈希功能
test_password = "my_secure_password_123"

# 生成哈希
hashed = get_password_hash(test_password)

print(f"📝 原始密码: {test_password}")
print(f"🔒 哈希结果: {hashed}")
print(f"\n🔍 哈希分析:")
print(f"   - 总长度: {len(hashed)} 字符")
print(f"   - 前缀 '$2b$': bcrypt 版本标识")
print(f"   - 成本因子: {hashed.split('$')[2]} (2^{hashed.split('$')[2]} = {2**int(hashed.split('$')[2])} 轮)")

(trapped) error reading bcrypt version
Traceback (most recent call last):
  File "/home/tomy/blog_fr/backend/.venv/lib/python3.13/site-packages/passlib/handlers/bcrypt.py", line 620, in _load_backend_mixin
    version = _bcrypt.__about__.__version__
              ^^^^^^^^^^^^^^^^^
AttributeError: module 'bcrypt' has no attribute '__about__'


📝 原始密码: my_secure_password_123
🔒 哈希结果: $2b$12$9xqIBtGcVrMa4FI2QxL3ReEME.i.vjHwQKMbGYOvFMHT52nnZ1ERq

🔍 哈希分析:
   - 总长度: 60 字符
   - 前缀 '$2b$': bcrypt 版本标识
   - 成本因子: 12 (2^12 = 4096 轮)


In [ ]:
# 验证每次哈希都会产生不同的结果（因为盐值不同）
hash1 = get_password_hash(test_password)
hash2 = get_password_hash(test_password)
hash3 = get_password_hash(test_password)

print("🧪 相同密码的多次哈希结果:")
print(f"   哈希1: {hash1}")
print(f"   哈希2: {hash2}")
print(f"   哈希3: {hash3}")
print(f"\n✅ 每次都不同? {hash1 != hash2 != hash3}")

## ✅ 密码验证测试

测试 `verify_password()` 函数，验证明文密码与哈希密码是否匹配。

In [ ]:
# 测试密码验证功能
correct_password = "my_secure_password_123"
wrong_password = "wrong_password"
stored_hash = get_password_hash(correct_password)

print("🔍 密码验证测试:")
print(f"   存储的哈希: {stored_hash[:30]}...")
print()

# 测试正确密码
result_correct = verify_password(correct_password, stored_hash)
print(f"   正确密码验证: {'✅ 通过' if result_correct else '❌ 失败'}")

# 测试错误密码
result_wrong = verify_password(wrong_password, stored_hash)
print(f"   错误密码验证: {'❌ 拒绝' if not result_wrong else '⚠️ 意外通过'}")

# 验证不同哈希但相同密码也能通过
result_with_different_hash = verify_password(correct_password, hash1)
print(f"   不同哈希验证: {'✅ 通过' if result_with_different_hash else '❌ 失败'}")

## 🎫 JWT 令牌生成测试

测试 `create_access_token()` 函数，创建 JWT 访问令牌。

In [ ]:
# 测试 Token 生成
user_id = "user_12345"
expires_delta = timedelta(minutes=settings.ACCESS_TOKEN_EXPIRE_MINUTES)

token = create_access_token(subject=user_id, expires_delta=expires_delta)

print(f"👤 用户 ID: {user_id}")
print(f"⏰ 过期时间: {settings.ACCESS_TOKEN_EXPIRE_MINUTES} 分钟")
print(f"\n🎫 生成的 Token:")
print(token)
print(f"\n📏 Token 长度: {len(token)} 字符")

In [ ]:
# 解码并验证 Token 内容
decoded = jwt.decode(token, settings.SECRET_KEY, algorithms=[ALGORITHM])

print("📋 Token 解码结果:")
print(f"   sub (用户标识): {decoded['sub']}")
print(f"   exp (过期时间): {decoded['exp']}")

# 转换为可读时间
from datetime import datetime, timezone
exp_time = datetime.fromtimestamp(decoded['exp'], tz=timezone.utc)
print(f"   过期时间 (可读): {exp_time.strftime('%Y-%m-%d %H:%M:%S')} UTC")

## 🔄 完整认证流程演示

模拟用户注册和登录的完整流程。

In [ ]:
# 模拟用户数据库
class FakeUserDB:
    """模拟用户数据库"""
    def __init__(self):
        self.users = {}

    def register(self, email: str, password: str) -> dict:
        """注册新用户"""
        if email in self.users:
            raise ValueError("用户已存在")

        user_id = f"user_{len(self.users) + 1}"
        hashed_password = get_password_hash(password)

        self.users[email] = {
            "id": user_id,
            "email": email,
            "hashed_password": hashed_password
        }
        return {"id": user_id, "email": email}

    def get_user(self, email: str) -> dict | None:
        """获取用户信息"""
        return self.users.get(email)

# 创建数据库实例
db = FakeUserDB()
print("✅ 模拟数据库已创建")

In [ ]:
# 步骤 1: 用户注册
print("=" * 50)
print("📝 步骤 1: 用户注册")
print("=" * 50)

registration_email = "alice@example.com"
registration_password = "alice_password_123"

new_user = db.register(registration_email, registration_password)
print(f"✅ 注册成功!")
print(f"   用户 ID: {new_user['id']}")
print(f"   邮箱: {new_user['email']}")

# 查看存储的哈希密码
stored_user = db.get_user(registration_email)
print(f"\n🔒 数据库中存储的哈希密码:")
print(f"   {stored_user['hashed_password']}")

In [ ]:
# 步骤 2: 用户登录
print("=" * 50)
print("🔐 步骤 2: 用户登录")
print("=" * 50)

def login(email: str, password: str) -> str | None:
    """模拟登录流程"""
    user = db.get_user(email)
    if not user:
        print("❌ 用户不存在")
        return None

    if not verify_password(password, user["hashed_password"]):
        print("❌ 密码错误")
        return None

    # 创建访问令牌
    token = create_access_token(
        subject=user["id"],
        expires_delta=timedelta(minutes=settings.ACCESS_TOKEN_EXPIRE_MINUTES)
    )
    print("✅ 登录成功!")
    return token

# 测试登录
access_token = login(registration_email, registration_password)
if access_token:
    print(f"\n🎫 Access Token:")
    print(f"   {access_token[:50]}...")

In [ ]:
# 步骤 3: 使用 Token 访问受保护的资源
print("=" * 50)
print("🛡️ 步骤 3: 验证 Token 访问受保护资源")
print("=" * 50)

def verify_token_and_get_user(token: str) -> dict | None:
    """验证 Token 并返回用户信息"""
    try:
        payload = jwt.decode(token, settings.SECRET_KEY, algorithms=[ALGORITHM])
        user_id = payload.get("sub")

        if user_id is None:
            print("❌ Token 中没有用户标识")
            return None

        print(f"✅ Token 验证成功!")
        print(f"   用户 ID: {user_id}")
        return {"user_id": user_id, "payload": payload}

    except jwt.ExpiredSignatureError:
        print("❌ Token 已过期")
        return None
    except jwt.InvalidTokenError as e:
        print(f"❌ 无效的 Token: {e}")
        return None

# 验证之前获取的 Token
result = verify_token_and_get_user(access_token)
if result:
    print(f"\n📋 完整 Payload: {result['payload']}")

In [ ]:
# 测试错误的登录尝试
print("=" * 50)
print("🧪 测试错误登录尝试")
print("=" * 50)

print("\n1. 测试错误密码:")
login(registration_email, "wrong_password")

print("\n2. 测试不存在的用户:")
login("nonexistent@example.com", "any_password")

## 📊 性能测试

测试密码哈希的计算时间（bcrypt 故意设计为较慢，以防止暴力破解）。

In [ ]:
import time

# 测试密码哈希时间
test_count = 5
passwords = [f"test_password_{i}" for i in range(test_count)]

print(f"⏱️ 测试 {test_count} 次密码哈希的耗时:")

start_time = time.time()
hashes = [get_password_hash(p) for p in passwords]
hash_time = time.time() - start_time

print(f"   总耗时: {hash_time:.3f} 秒")
print(f"   平均每次: {hash_time/test_count*1000:.1f} 毫秒")

# 测试密码验证时间
print(f"\n⏱️ 测试 {test_count} 次密码验证的耗时:")

start_time = time.time()
results = [verify_password(passwords[i], hashes[i]) for i in range(test_count)]
verify_time = time.time() - start_time

print(f"   总耗时: {verify_time:.3f} 秒")
print(f"   平均每次: {verify_time/test_count*1000:.1f} 毫秒")

print(f"\n💡 bcrypt 故意设计较慢，使暴力破解变得不切实际")
print(f"   如果攻击者每秒只能尝试 {1/(hash_time/test_count):.0f} 次，")
print(f"   破解一个 8 位复杂密码需要天文数字的时间")

## 📚 总结

本 notebook 测试了 `app/core/security.py` 模块的三个核心功能：

| 函数 | 功能 | 使用场景 |
|------|------|----------|
| `get_password_hash()` | 生成密码哈希 | 用户注册 |
| `verify_password()` | 验证密码 | 用户登录 |
| `create_access_token()` | 创建 JWT Token | 登录成功后颁发令牌 |

### 安全要点：

1. **密码永远不要明文存储** - 使用 bcrypt 哈希
2. **每次哈希都使用不同的盐值** - 防止彩虹表攻击
3. **Token 应该有合理的过期时间** - 减少被盗用的风险
4. **SECRET_KEY 必须保密** - 从环境变量读取，不要硬编码